# Chatbot for game recommandation

### Import

In [1]:
!pip install langchain
!pip install langchain_openai

In [2]:
!pip install --upgrade typing_extensions

In [3]:
!pip uninstall openai langchain langchain_openai -y
!pip install openai langchain langchain_openai --upgrade

Found existing installation: openai 1.61.0
Uninstalling openai-1.61.0:
  Successfully uninstalled openai-1.61.0
Found existing installation: langchain 0.3.17
Uninstalling langchain-0.3.17:
  Successfully uninstalled langchain-0.3.17
Found existing installation: langchain-openai 0.3.3
Uninstalling langchain-openai-0.3.3:
  Successfully uninstalled langchain-openai-0.3.3
   ---------------------------------------- 0.0/606.1 kB ? eta -:--:--
   ----------------- ---------------------- 262.1/606.1 kB ? eta -:--:--
   ---------------------------------------- 606.1/606.1 kB 2.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.33
    Uninstalling langchain-core-0.3.33:
      Successfully uninstalled langchain-core-0.3.33
  Attempting uninstall: langchain-text-splitters
    Found existing inst

In [4]:
!pip show typing_extensions

Name: typing_extensions
Version: 4.12.2
Summary: Backported and Experimental Type Hints for Python 3.8+
Home-page: 
Author: 
Author-email: "Guido van Rossum, Jukka Lehtosalo, Łukasz Langa, Michael Lee" <levkivskyi@gmail.com>
License: 
Location: C:\Users\salla\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: 
Required-by: Faker, fastapi, gradio, gradio_client, gymnasium, huggingface-hub, langchain-core, openai, optree, pydantic, pydantic_core, SQLAlchemy, tensorflow_intel, torch, twitchio, typer


In [5]:
import requests
import time
import json
import numpy as np
from scipy.spatial.distance import cosine
from sentence_transformers import SentenceTransformer
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

In [6]:
from sentence_transformers import SentenceTransformer
import json

embedding_model = SentenceTransformer('all-MiniLM-L6-v2', cache_folder="./cache")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\salla\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\salla\OneDrive\Documents\Cours\A3\NLP_and_Text_Mining\Chatbot\cache\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Charger les jeux IGDB
with open("igdb_games_embedding.json", "r") as f:
    games = json.load(f)

### Sentiment analysis

The sentiment analysis is used to detect whether the user liked a game and would like a similar game or if the user disliked a game and would prefer a different game.
Therefore the taks is a binary classification.
It will be done by training a logistic regression on a dataset of steam game reviews. This dataset will be preprocessed with tfidf to create the features.

This is a basic implementation of sentiment analysis because we wanted to start with easier solutions first before using complicated tools.

,id,app_id,content,author_id,sentiment
0,181331361,100,At least its a counter strike -1/100,76561199556485100,Negative


,content,sentiment
0,At least its a counter strike,negative
1,Uh So far my playthrough has not been great...,negative
2,Better mechanics than cs,negative
3,buggy mess and NOT fun to play at all,negative
4,Whoever came up with this is gonna fucking ge...,negative


(201151, 97755)

0.8414532293986637

('positive', 0.9238021237514981)

In [47]:
# Test the sentiment analysis for out task
texts = [
    "I like Minecraft, what similar game should I play?", 
    "I don't like this game, do you have any other recommendations ?",
    "Recommend me a game similar to Hades.",
    "I am doing wonderful today. I would like a game different from this one.",
    "I love this game, it's so fun but could I get a different game to try ?",
]

for text in texts:
    print(predict_sentiment(text), text)

('positive', 0.7654977693539498) I like Minecraft, what similar game should I play?
('negative', 0.7792993970239463) I don't like this game, do you have any other recommendations ?
('positive', 0.6766948054523736) Recommend me a game similar to Hades.
('positive', 0.8712335244563272) I am doing wonderful today. I would like a game different from this one.
('positive', 0.9638609423410824) I love this game, it's so fun but could I get a different game to try ?


### Fonction de recommandation

In [48]:
import numpy as np
from scipy.spatial.distance import cosine

# Fonction de recommandation basée sur la similarité
def recommend_game(user_query):

    
    # Find the most similar games with embeddings
    query_embedding = embedding_model.encode(user_query)
    similarities = [
        (game["name"], 1 - cosine(query_embedding, game["embedding"]))
        for game in games if "embedding" in game
    ]

    

# Test
user_input = "I'm looking for an RPG with a good storyline and deep characters."
print("Jeux recommandés :", recommend_game(user_input))


Jeux recommandés : []


In [7]:
#!pip install langchain langchain_openai "pydantic<2" openai
#!pip install --upgrade typing_extensions

In [8]:
class ChatSession:
    def __init__(self, memsize, llm):
        self.llm = llm
        self.memsize = memsize
        self.history = []
        self.llmID = "ASSISTANT:"
        self.userID = "USER:"
                
    def summarizeStrategy(self):
        summaryPrompt = "Résume la conversation suivante en gardant un maximum d'informations fournies par " + self.userID + " :\n"
        lastMessage, lastMessageLength = self.history[-1]
        history = ""
        for m, l in self.history[:-1]:
            history += m + "\n"
        summary = self.llm.invoke([HumanMessage(content=summaryPrompt + "\n" + history)]).content
        summaryLength = self.estimateLength(summary)
        if summaryLength + lastMessageLength > self.memsize:
            self.fifoStrategy()
            return
        self.history = [(summary, summaryLength), (lastMessage, lastMessageLength)]
    
    def fifoStrategy(self):
        while self.historyLength() > self.memsize:
            self.history = self.history[1:]

    def historyLength(self):
        return sum(length for _, length in self.history)

    def estimateLength(self, text):
        return self.llm.get_num_tokens(text)

    def addToHistory(self, prompt):
        self.history.append((prompt, self.estimateLength(prompt)))
        if self.historyLength() > self.memsize:
            self.summarizeStrategy()

    def chat(self, prompt):
        self.addToHistory(self.userID + " " + prompt)

        recommendations = recommend_game(prompt)
        games_list = ", ".join([game[0] for game in recommendations])

        chat_prompt = f"""
        L'utilisateur cherche un jeu correspondant à : "{prompt}".
        Voici les jeux recommandés : {games_list}.
        Formule une réponse naturelle et engageante.
        """

        fullPrompt = 'Historique de conversation :\n'
        for msg, _ in self.history:
            fullPrompt += msg + "\n"

        fullPrompt += "\n" + chat_prompt

        resp = self.llm.invoke([HumanMessage(content=fullPrompt)])
        content = resp.content
        if content.startswith(self.llmID + ": "):
            content = content[len(self.llmID + ": "):]

        self.addToHistory(self.llmID + " " + content)
        return content

## Initialisation du LLM

In [ ]:
llm = ChatOpenAI(
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="mistralai/mistral-small-3.1-24b-instruct:free",
)

In [16]:
CONTEXT_LENGTH = 500  
session = ChatSession(CONTEXT_LENGTH, llm)

In [ ]:
print("\n Bienvenue dans le chatbot de recommandation de jeux vidéo avec mémoire !")
print("\n Pose-moi une question (ex: 'Je veux un RPG avec un bon scénario').")
print("\n Tape 'exit' pour quitter.\n")

while True:
    user_input = input("👤 Vous : ")
    if user_input.lower() == "exit":
        print("À bientôt !")
        break
    
    response = session.chat(user_input)
    print(f"🤖 Bot : {response}\n")



 Bienvenue dans le chatbot de recommandation de jeux vidéo avec mémoire !

 Pose-moi une question (ex: 'Je veux un RPG avec un bon scénario').

 Tape 'exit' pour quitter.

👤 Vous :  I want to have an adventure game with blades and blood
🤖 Bot : It sounds like you're in the mood for an exciting adventure filled with action and intensity! Based on your preferences, here are a few games that might suit you perfectly:

1. **Blood & Blade**: This game is known for its thrilling combat and rich storyline, making it a great choice for fans of adventure and blade-wielding action.

2. **Escape from the Vampire's Tomb**: If you enjoy a mix of mystery and adventure with a touch of the supernatural, this could be a fantastic pick. It's got all the right ingredients for an adrenaline-pumping experience.

3. **Spellcasting 201: The Sorcerer's Appliance**: While this game leans more into magic and spellcasting, it still offers plenty of adventure and excitement, especially if you enjoy a blend of tr